# CLEANING YAWDD DATASET

In [1]:
import os
import shutil

# Your specific path
raw_data_path = r'D:\FYP_ARMAN\Driver-Fatigue-Detection-Using-Vision-Based-Machine-Learning\YAWDD_DATASET'
processed_path = 'data/sorted_yawdd'

for root, dirs, files in os.walk(raw_data_path):
    for file in files:
        if file.endswith(".avi"):
            # 1. Identify Camera Angle (Dashboard vs Mirror)
            # Your path has "Dash", so we check for that
            angle = 'dash' if 'Dash' in root else 'mirror'
            
            # 2. Identify Label (Yawning vs Normal) 
            # YawDD filenames usually contain 'Yawning' or 'NoGlasses'/'Glasses' (Normal)
            if 'Yawning' in file:
                label = 'yawn'
            else:
                label = 'normal'
            
            # 3. Organize into a structure for CNN/VGG16 training
            target_dir = os.path.join(processed_path, angle, label)
            os.makedirs(target_dir, exist_ok=True)
            
            source_file = os.path.join(root, file)
            # Note: Using copy2 to preserve metadata
            shutil.copy2(source_file, os.path.join(target_dir, file))

print("Organization complete. Files sorted by Angle and Class.")

Organization complete. Files sorted by Angle and Class.


# Frame Extraction & Feature Analysis

In [2]:
import mediapipe as mp
print(f"Version: {mp.__version__}")
print(f"File Path: {mp.__file__}")

Version: 0.10.32
File Path: c:\Users\arman\miniconda3\envs\FYP\Lib\site-packages\mediapipe\__init__.py


In [6]:
import cv2
import dlib
import os
import numpy as np

# 1. Initialize Dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# 2. Define Windows-specific paths
base_path = r'data\sorted_yawdd'
output_path = r'data\processed_images'

def get_mar(shape):
    # Implements the MAR formula from your literature review 
    # Vertical distances between inner lips
    v1 = np.linalg.norm(shape[61] - shape[67])
    v2 = np.linalg.norm(shape[63] - shape[65])
    # Horizontal distance
    h = np.linalg.norm(shape[60] - shape[64])
    return (v1 + v2) / (2.0 * h)

# 3. Automation Loop
for root, dirs, files in os.walk(base_path):
    for file in files:
        if not file.endswith('.avi'): continue
        
        video_path = os.path.join(root, file)
        cap = cv2.VideoCapture(video_path)
        frame_idx = 0
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break
            
            # Sample every 5th frame to manage computational resources [cite: 197]
            if frame_idx % 5 == 0:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = detector(gray)
                
                for face in faces:
                    landmarks = predictor(gray, face)
                    shape = np.array([[landmarks.part(i).x, landmarks.part(i).y] for i in range(68)])
                    
                    mar = get_mar(shape)
                    
                    # Logic: Extract yawning frames or all normal frames [cite: 287]
                    if ('yawn' in root and mar > 0.3) or ('normal' in root):
                        # Resize to 224x224 for VGG16/ResNet-50/MobileNetV4 [cite: 218]
                        final_img = cv2.resize(frame, (224, 224))
                        
                        save_dir = os.path.join(output_path, os.path.relpath(root, base_path))
                        os.makedirs(save_dir, exist_ok=True)
                        cv2.imwrite(os.path.join(save_dir, f"{file}_f{frame_idx}.jpg"), final_img)
            
            frame_idx += 1
        cap.release()

print("Dlib extraction complete! Your YawDD dataset is now processed for training.")

RuntimeError: Unable to open shape_predictor_68_face_landmarks.dat